<h1> A1: Predicting Car Price</h1>

In this assignment, I will solve a problem, i.e., Chaky company makes some car but he has difficulty setting the price for the car. I will make a simple web-based car price prediction system.

This is a regression problem trying to predict car price.

<ul>The followings describe the feautres:
<br></br>
<b>
<li>name
<li>year
<li>selling_price
<li>km_driven
<li>fuel
<li>seller_type
<li>transmission
<li>owner
<li>mileage
<li>engine
<li>max_power
<li>torque
<li>seats
</b>
</ul>

<h2> Task1. Preparing the datasets </h2>

<h3> 1. Importing Libraries </h3>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
np.__version__, pd.__version__, sns.__version__, matplotlib.__version__, sns.__version__

('1.24.4', '1.5.3', '0.12.2', '3.7.2', '0.12.2')

<h3>1. Load Data</h3>

In [3]:
#read csv
df = pd.read_csv('/root/.vscode-server/data/lab1_Cars.csv')

In [4]:
#print the first row of data
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [5]:
#check the shape of the data
df.shape

(8128, 13)

In [6]:
#statistical info Hint:
df.describe()

,year,selling_price,km_driven,seats
count,8128.000000,8.128000e+03,8.128000e+03,7907.000000
mean,2013.804011,6.382718e+05,6.981951e+04,5.416719
std,4.044249,8.062534e+05,5.655055e+04,0.959588
min,1983.000000,2.999900e+04,1.000000e+00,2.000000
25%,2011.000000,2.549990e+05,3.500000e+04,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,5.000000
75%,2017.000000,6.750000e+05,9.800000e+04,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,14.000000


In [7]:
#check data types of the input data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [8]:
#check the column names
df.columns

Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats'],
      dtype='object')

<h3>2. Exploratory Data Analysis</h3>

EDA is an essential step to inspect the data, so to better understand nature of the given data.

<h4> Renaming </h4>

In [9]:
#Rename "Name" Column to "brand", I will not rename for the rest columns as it is already perfect.
df.rename (
    columns= {'name' : 'brand'}, inplace = True
)

In [10]:
#check the columns name
df.columns

Index(['brand', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats'],
      dtype='object')

<h4> Label Encoding </h4>

Now I would like to change feature "owner" values to numerical values by mapping <b> First Owner = 1, ..., Test Drive Cars = 5

In [11]:
#check the owner column unique value
df['owner'].unique()

array(['First Owner', 'Second Owner', 'Third Owner',
       'Fourth & Above Owner', 'Test Drive Car'], dtype=object)

In [12]:
#import labelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['owner'] = le.fit_transform (df['owner'])
df['owner'].unique()

array([0, 2, 4, 1, 3])

Again, I will do label encoding for "transmission" feature

In [13]:
#check for my array
df['transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [14]:
df['transmission'] = le.fit_transform(df['transmission'])
df['transmission'].unique()

array([1, 0])

<h4> Other Preparation </h4>

For feature "Fuel" >> I will remove all rows with 'CNG' and 'LPG'

In [15]:
df.shape

(8128, 13)

In [16]:
# Delete rows where 'fuel' == 'CNG'
# Using drop() to delete rows based on column value
df.drop(df[df['fuel'] == 'CNG'].index, inplace = True)

In [17]:
#check after dropping CNG
df.shape

(8071, 13)

In [18]:
# Delete rows where 'fuel' == 'LPG'
# Using drop() to delete rows based on column value
df.drop(df[df['fuel'] == 'LPG'].index, inplace = True)

In [19]:
#check after dropping LPG
df.shape

(8033, 13)

For feature 'mileage', I will remove 'kmpl' using df.mileage.str.split and 
convert column to numerical type (e.g. float)

In [20]:
#usage of split
# series.str.split (pad = None, *, n = -1, expand = flase, regex = None)

#df with split value of columns (set expand = True: it will return a df after splitted the values; n=1: total separations will be 1 time)
df1 = df["mileage"].str.split (" ",n = 1, expand = True)

#making the splitted 0 column as 'mileage' features
df['mileage'] = df1[0]

In [21]:
#check the datatype of a series
df['mileage']

0        23.4
1       21.14
2        17.7
3        23.0
4        16.1
        ...  
8123     18.5
8124     16.8
8125     19.3
8126    23.57
8127    23.57
Name: mileage, Length: 8033, dtype: object

In [22]:
#convert 'mileage' feature DType to float
df['mileage']=df['mileage'].astype('float64')

In [23]:
df['mileage']

0       23.40
1       21.14
2       17.70
3       23.00
4       16.10
        ...  
8123    18.50
8124    16.80
8125    19.30
8126    23.57
8127    23.57
Name: mileage, Length: 8033, dtype: float64

For feature 'engine', I will remove 'CC' and convert it datatype to numerical (float) 

In [24]:
#check feature 'engine'
df['engine']

0       1248 CC
1       1498 CC
2       1497 CC
3       1396 CC
4       1298 CC
         ...   
8123    1197 CC
8124    1493 CC
8125    1248 CC
8126    1396 CC
8127    1396 CC
Name: engine, Length: 8033, dtype: object

In [25]:
#remove CC of engine and put the results into temporary dataframe
df2 = df['engine'].str.split (' ', n = 1, expand = True)
df2

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [26]:
#update the value: update the resulted df[0] to the original df
df['engine']  = df2[0]

#check the df['engine']
df['engine']

0       1248
1       1498
2       1497
3       1396
4       1298
        ... 
8123    1197
8124    1493
8125    1248
8126    1396
8127    1396
Name: engine, Length: 8033, dtype: object

In [27]:
#convert dtype of 'engin' to float
df['engine'] = df['engine'].astype('float64')

For feature 'maxpower', I will repeat the same steps as what I did to feature 'engine'

In [28]:
#check 'maxpower'
df['max_power']

0           74 bhp
1       103.52 bhp
2           78 bhp
3           90 bhp
4         88.2 bhp
           ...    
8123     82.85 bhp
8124       110 bhp
8125      73.9 bhp
8126        70 bhp
8127        70 bhp
Name: max_power, Length: 8033, dtype: object

In [29]:
#remove bhp
df3 = df['max_power'].str.split (" ", n = 1, expand = True)
df3

,0,1
0,74,bhp
1,103.52,bhp
2,78,bhp
3,90,bhp
4,88.2,bhp
...,...,...
8123,82.85,bhp
8124,110,bhp
8125,73.9,bhp
8126,70,bhp


In [30]:
#replace
df['max_power'] = df3[0]
df['max_power']

0           74
1       103.52
2           78
3           90
4         88.2
         ...  
8123     82.85
8124       110
8125      73.9
8126        70
8127        70
Name: max_power, Length: 8033, dtype: object

In [31]:
#change dtype of max_power
df['max_power'] = df['max_power'].astype ("float64")
df['max_power']

0        74.00
1       103.52
2        78.00
3        90.00
4        88.20
         ...  
8123     82.85
8124    110.00
8125     73.90
8126     70.00
8127     70.00
Name: max_power, Length: 8033, dtype: float64

For feature 'brand', I will take only the first word

In [32]:
#check
df['brand'].unique()

array(['Maruti Swift Dzire VDI', 'Skoda Rapid 1.5 TDI Ambition',
       'Honda City 2017-2020 EXi', ..., 'Tata Nexon 1.5 Revotorq XT',
       'Ford Freestyle Titanium Plus Diesel BSIV',
       'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV'], dtype=object)

In [33]:
#split
df4 = df['brand'].str.split (" ", n = 1, expand = True)
df4

,0,1
0,Maruti,Swift Dzire VDI
1,Skoda,Rapid 1.5 TDI Ambition
2,Honda,City 2017-2020 EXi
3,Hyundai,i20 Sportz Diesel
4,Maruti,Swift VXI BSIII
...,...,...
8123,Hyundai,i20 Magna
8124,Hyundai,Verna CRDi SX
8125,Maruti,Swift Dzire ZDi
8126,Tata,Indigo CR4


In [34]:
df['brand'] = df4[0]
df['brand'].unique()

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

Drop the feature "torque"

In [35]:
#drop
df = df.drop (columns = ['torque'])

#check the columns
df.columns

Index(['brand', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats'],
      dtype='object')

I will remove some samples of which 'owner' == test driver car

In [36]:
df.shape

#check my df['owner'] sample to make sure it is correct or not
df[df['owner'] == 3]

,brand,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
4383,Volkswagen,2019,1350000,5400,Diesel,Dealer,1,3,22.27,1498.0,108.60,5.0
4950,Audi,2019,6223000,7800,Petrol,Dealer,0,3,15.26,1798.0,187.74,5.0
4951,Audi,2019,5923000,11500,Petrol,Dealer,0,3,15.26,1798.0,187.74,5.0
4952,Audi,2019,6523000,23600,Petrol,Dealer,0,3,15.26,1798.0,187.74,5.0
6220,Honda,2019,2000000,24857,Petrol,Dealer,0,3,16.50,1799.0,139.46,5.0


In [37]:
#drop the samples for test drive car
df.drop(df[df['owner'] == 3].index, inplace = True)

In [38]:
#check the shape
df.shape

# df[df['owner'] == 3]

(8028, 12)

For 'sellingprice', I will transform using the log transform as it is a big number

In [39]:
#check selling_price
df['selling_price']

0       450000
1       370000
2       158000
3       225000
4       130000
         ...  
8123    320000
8124    135000
8125    382000
8126    290000
8127    290000
Name: selling_price, Length: 8028, dtype: int64

In [40]:
#log_transform for selling_price
df['selling_price'] = np.log(df['selling_price'])
df['selling_price']


0       13.017003
1       12.821258
2       11.970350
3       12.323856
4       11.775290
          ...    
8123    12.676076
8124    11.813030
8125    12.853176
8126    12.577636
8127    12.577636
Name: selling_price, Length: 8028, dtype: float64

In [41]:
#check the final dataframe
df

,brand,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
0,Maruti,2014,13.017003,145500,Diesel,Individual,1,0,23.40,1248.0,74.00,5.0
1,Skoda,2014,12.821258,120000,Diesel,Individual,1,2,21.14,1498.0,103.52,5.0
2,Honda,2006,11.970350,140000,Petrol,Individual,1,4,17.70,1497.0,78.00,5.0
3,Hyundai,2010,12.323856,127000,Diesel,Individual,1,0,23.00,1396.0,90.00,5.0
4,Maruti,2007,11.775290,120000,Petrol,Individual,1,0,16.10,1298.0,88.20,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai,2013,12.676076,110000,Petrol,Individual,1,0,18.50,1197.0,82.85,5.0
8124,Hyundai,2007,11.813030,119000,Diesel,Individual,1,1,16.80,1493.0,110.00,5.0
8125,Maruti,2009,12.853176,120000,Diesel,Individual,1,0,19.30,1248.0,73.90,5.0
8126,Tata,2013,12.577636,25000,Diesel,Individual,1,0,23.57,1396.0,70.00,5.0


<h5>2.1 Univariate Analysis</h5>

Single variable exploratory data analysis

<h5>Couontplot</h5>

In [ ]:
sns.countplot (data = df, x = 'selling_price')

In [ ]:
sns.countplot (data = df, x = 'fuel')

In [ ]:
sns.countplot (data = df, x = 'seller_type')

In [ ]:
sns.countplot (data = df, x = 'transmission')

In [ ]:
sns.countplot (data = df, x = 'owner')

In [ ]:
sns.countplot (data = df, x = 'seats')

<h5>Distribution Plot</h5>

In [ ]:
sns.displot(data = df, x = 'engine')

In [ ]:
sns.displot(data = df, x = 'year')

<h5>2.2 Multivariate analysis</h5>

Multiple variable exploratory data analysis

<h5>Boxplot</h5>

In [ ]:
sns.boxplot (x = df['year'], y = df['brand']);
plt.ylabel("Brand")
plt.xlabel("Year")

In [ ]:
sns.boxplot (x = df['engine'], y = df['fuel']);
plt.xlabel("Engine")
plt.ylabel("Fuel")


In [ ]:
sns.boxplot (x = df['seats'], y = df['selling_price']);
plt.xlabel("seats")
plt.ylabel("selling price")

In [ ]:
sns.boxplot (x = df['fuel'], y = df['selling_price']);
plt.xlabel("fuel")
plt.ylabel("selling price")

In [ ]:
sns.boxplot (x = df['seller_type'], y = df['selling_price']);
plt.xlabel("seller_type")
plt.ylabel("selling_price")

<h5> Scatterplot </h5>

In [ ]:
sns.scatterplot (x = df ['year'], y = df['selling_price'], hue = df['fuel'])
#diesel car are more expensive than petrol car

In [ ]:
sns.scatterplot (x = df ['year'], y = df['selling_price'], hue = df['seller_type'])

In [ ]:
sns.scatterplot (x = df ['selling_price'], y = df['brand'], hue = df['seats'])

In [ ]:
sns.scatterplot (x = df ['year'], y = df['selling_price'], hue = df['seats'])

<h5>Correlation Matrix</h5>

Correlation matrix is used to find strong factors predicting the life expectancy. It's also for checking features are too correlated.

In [ ]:
#Check out heatmap
plt.figure (figsize = (15,8))
my_df = df.select_dtypes (exclude = [object])
sns.heatmap (my_df.corr(), annot = True, cmap = 'coolwarm')

<h4>Predictive Power Score</h4>

In [ ]:
import ppscore as pps

#before using pps

In [ ]:

#ppscore_df = df.select_dtypes (exclude = [object])
matrix_df = pps.matrix(df)[['x','y','ppscore']].pivot (columns='x', index = 'y', values= 'ppscore')

#plot
plt.figure (figsize = (15,8))
sns.heatmap (matrix_df, vmin = 0, vmax = 1, cmap = "Blues", linewidths = 0.5, annot = True)

<h3>3. Feature Engineering</>

<h3>4. Feature Selection</h3>

In [ ]:
#x is strong features
# X = df[['transmission','max_power']]

#y is the selling_price label
# y = df['selling_price']


<h5>Train Test Split</h5>

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split (X,y, test_size = 0.3, random_state = 16)

<h4>Preprocessing</h4>

<h5>Null Values</h5>

In [ ]:
#check for null vaules in train set
# X_train[['transmission','max_power']].isna().sum()

In [ ]:
#check for null values in test set
# X_test[['transmission','max_power']].isna().sum()

In [ ]:
# y_train.isna().sum()

In [ ]:
# y_test.isna().sum()

In [ ]:
# sns.displot (data=df, x='transmission')

In [ ]:
# sns.displot(data = df, x = 'max_power')

In [ ]:
# sns.displot(y_train)

In [ ]:
# sns.displot(y_test)

In [ ]:
#After checking the null values for my features: 'transmission' and 'max_power', only 'max_power' feature has the null values.
# for the max_power, I will fill with median value, because we already 
#syntax df.fillna (value = none, method=none, axis = none, inplace = false, limit = none, downcase = none)
#X_train['max_power'].fillna ()

# X_train['max_power'].fillna(X_train['max_power'].median(),inplace = True)
# X_test['max_power'].fillna(X_train['max_power'].median(),inplace = True)

#for our label that is selling_price, there is no null value.

In [ ]:
#check
# X_train.isna().sum()

In [ ]:
#check
# X_test.isna().sum()

In [ ]:
# noneed
# df.describe()

<h5>Checking Outliers</h5>

In [ ]:
# #Create a dictionary of columns
# col_dict = {'transmission':1, 'max_power':2}

# #detect outliers in each variable using box plots
# plt.figure(figsize=(20,30))
           
# for variable,i in col_dict.items():
#     plt.subplot(5,4,i)
#     plt.boxplot(X_train[variable])
    
#     plt.title(variable)
# plt.show()
    

In [ ]:
# def outlier_count(col, data = X_train):
    
#     #calculate 25% quatile and 75% quatile
#     q75, q25 = np.percentile (data[col],[75,25])
    
#     #calculate your inter quatile
#     iqr = q75-q25
    
#     #min_val and max_val
#     min_val = q25 - (iqr * 1.5)
#     max_val = q75 + (iqr * 1.5)
    
#     #count number of outliers, which are the data that are less than min_val and more than max_val calculated above
#     outlier_count = len(np.where((data[col] > max_val) | (data[col] < min_val))[0])
    
#     #calculate the percentage of the outliers
#     outlier_percent = round (outlier_count/len(data[col])*100,2)
    
#     if(outlier_count > 0):
#         print("\n" + 15*"-" + col + 15*'-' + "\n")
#         print ('Number of outliers: {}'.format (outlier_count))
#         print ('Percentage of data that is outlier : {}'.format(outlier_percent))

In [ ]:
# for col in X_train.columns:
#     outlier_count(col)

<h3>Scaling </h3>

In [ ]:
# from sklearn.preprocessing import StandardScaler

# #feature scaling helps improve reach convergence faster
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform (X_test)

In [ ]:
# #check the shape of all X_train, y_train, y_test
# print ("Shape of X_train: ", X_train.shape)
# print ("Shape of X_test: ", X_test.shape)
# print ("Shape of y_train: ", y_train.shape)
# print ("Shape of y_test: ", y_test.shape)


<h3> Modelling </h3>

<p>
    IMPORTANT NOTES FOR USNIGN SKLEARN:<br>
    define some algorithms and compare them using cross-validation: using scikit-learn which provide a huge pool of ML algorithms. <BR>
    need to aware the data shape that sklearn wants <br>
    sklearn requires two inputs: X and y: <br>
    X is feature matrix, shape = [n_samples,n_features] and y is label vector,shape = [n_samples, ] or [n_samples, n_targets]<br>
</p>

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr = LinearRegression()
# lr.fit (X_train,y_train)
# yhat = lr.predict (X_test)

# print ("MSE: ", mean_squared_error (y_test, yhat))
# print ("r2: ", r2_score(y_test, yhat))

Cross validation + Grid serch

In [ ]:
#importing regression algorithms libraries
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#put the models that we will use into a list
algorithms = [LinearRegression(), SVR(), KNeighborsRegressor(), DecisionTreeRegressor(random_state=0), RandomForestRegressor(n_estimators = 100, random_state = 0)]

Do simple cross-validation.

In [ ]:
df.shape

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

#lists for keeping MSE
train_mse = []
test_mse = []

#defining number of splits using kfold
kfold = KFold (n_splits = 8, shuffle = True)

#finding the scores for all the algorithms
# # for i, model in enumerate (algorithms):
#     scores  = cross_val_score (model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
#     print(f"{algorithms[i]} - Score: {scores}; Mean: {scores.mean()}")


It seems random forest regressor do very well. So, I will do the grid search the find the best version of the model.

<h5> Grid Search </h5>

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid =  {'bootstrap' : [True], 'max_depth' : [5,10, None], 'n_estimators': [5,6,7,8,9,10,11,12,13,14,15]}

rf = RandomForestRegressor (random_state = 1)

grid = GridSearchCV (estimator=rf,
                     param_grid=param_grid,
                     cv=kfold,
                     n_jobs=-1,
                     return_train_score=True,
                     refit=True,
                     scoring='neg_mean_squared_error')

#fit your grid search
# grid.fit (X_train, y_train);

In [ ]:
grid.best_params_

In [ ]:
#best your gird_search's best score
best_mse = grid.best_score_

In [ ]:
best_mse

<h3> 7. Testing</h3>

In [ ]:
# yhat = grid.predict (X_test)
# mean_squared_error(y_test,yhat)

---------------------------------------------------------------------------------

<h3> Part 2 - 4. Feature Selection </h3>

In [ ]:
#feature selection
X2 = df[['engine','year']]
y2 = df['selling_price']

In [ ]:
#train test split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 0.3, random_state=4)

In [ ]:
X_train2.shape

In [ ]:
#preprocessing

#check for null values
X_train2[['engine','year']].isna().sum()

In [ ]:
X_test2[['engine','year']].isna().sum()

In [ ]:
y_train2.isna().sum()

In [ ]:
y_test2.isna().sum()

In [ ]:
sns.displot(data = df, x = 'engine')

In [ ]:
sns.displot (data = df, x = 'year')

In [ ]:
#fill for the engine
X_train2['engine'].fillna (X_train2['engine'].median(), inplace = True)
X_test2['engine'].fillna (X_train2['engine'].median(), inplace = True)

In [ ]:
X_train2[['engine', 'year']].isna().sum()

In [ ]:
#checking outliers
#create dictionary column
col_dict2 = {'engine':1, 'year':2}

plt.figure (figsize = (20,30))

for variable,i in col_dict2.items():
    plt.subplot (5,4,i)
    plt.boxplot (X_train2[variable])
    plt.title (variable)
plt.show()

In [ ]:
def outlier_count (col, data = X_train2):
    q752, q252 = np.percentile (data[col], [75, 25])
    
    iqr = q752-q252
    
    min_val2 = q252 - (iqr * 1.5)
    max_val2 = q752 + (iqr * 1.5)
    
    outlier_count2 = len(np.where ((data[col]> max_val2) | (data[col]<min_val2))[0])
    
    outlier_percentate2 = round (outlier_count2 / len(data[col])*100,2)
    
    if (outlier_count2 > 0):
        print ("\n" +15*'-' + col + 15*'-' + "\n")
        print ("Number of outliers: {}".format (outlier_count2))
        print ('Percentage of data that is outlier:{}'.format(outlier_percentate2))

In [ ]:
for col in X_train2.columns:
    outlier_count (col)

In [ ]:
#Scaling
scaler2 = StandardScaler()
X_train2 = scaler2.fit_transform(X_train2)
X_test2  = scaler2.fit_transform(X_test2)

In [ ]:
#Modelling
#list
train_mse2 = []
test_mse2 = []

for i, model in enumerate (algorithms):
    scores2 = cross_val_score (model, X_train2, y_train2, cv=kfold, scoring='neg_mean_squared_error')
    print (f"{algorithms[i]} - Scores: {scores2}; Mean: {scores2.mean()}")

In [ ]:
#grid_search
grid.fit (X_train2, y_train2);

In [ ]:
grid.best_params_

In [ ]:
best_mse = grid.best_score_
best_mse

In [ ]:
#testing
yhat2 = grid.predict(X_test2)
mean_squared_error (y_test2, yhat2)

<h2> 8. Analysis of Feature Importance </h2>

<h5>Algorithm way</h5>

In [ ]:
#stored in this variable
#random forest grid

rf = grid.best_estimator_
rf.feature_importances_

In [ ]:
#let's plot
plt.barh (X.columns, rf.feature_importances_)

In [ ]:
sorted_idx = rf.feature_importances_.argsort()
plt.barh (X.columns[sorted_idx], rf.feature_importances_[sorted_idx])
plt.xlabel ("RandomForestRegressor")

<h5> Permutation way </h5>

In [ ]:
from sklearn.inspection import permutation_importance

per_imp = permutation_importance(rf, X_test2, y_test2)

#plot
sorted_idx = per_imp.importances_mean.argsort()
plt.barh (X.columns[sorted_idx], per_imp.importances_mean[sorted_idx])
plt.xlabel ("RandomForestRegressor")

<h5> Shape Way </h5>

In [ ]:
import shap

explainer = shap.TreeExplainer (rf)
shap_values = explainer.shap_values (X_test2)

In [ ]:
#shap provides plot
shap.summary_plot (shap_values, X_test2, plot_type="bar", feature_names=X2.columns)

<h2> Inference </h2>

In [ ]:
import pickle

In [ ]:
#save the model to disk
filename = 'lab1_CarPricePrediction.model'
pickle.dump (grid, open(filename,'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load (open (filename, 'rb'))

In [ ]:
#try to create one example
# transform predicted y value back to original value
#df['selling_price'] = np.exp (df['selling_price'])
df[['engine','year','selling_price']].loc[2]

In [ ]:
sample = np.array([[109,2014]])

In [ ]:
predicted_car_price_1 = loaded_model.predict(sample)
predicted_car_price = np.exp (predicted_car_price_1)
predicted_car_price

In [ ]:
# predicted_car_price_1

4488301.339588621

<h2> Summary</h2>

- which features are important? why?

- which algorithm performs well? which does not? why?